In [1]:
!pip install kaggle

In [2]:
!pip install sqlalchemy


In [3]:
!pip install pyodbc

In [4]:
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [6]:
import pandas as pd
df = pd.read_csv('orders.csv.zip',na_values = ['Not Available','unknown'])
df['Ship Mode'].unique()


array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [7]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [8]:
df['discount']=df['list_price']*df['discount_percent']*.01

In [9]:
df['sale_price']=df['list_price']-df['discount']

In [10]:
df['profit'] = df['sale_price']-df['cost_price']

In [11]:
df['order_date']=pd.to_datetime(df['order_date'],format = "%Y-%m-%d" )

In [12]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [13]:
df.drop(columns=['list_price','cost_price','discount_percent'],inplace = True)

In [18]:
import sqlalchemy as sal
from sqlalchemy import create_engine

# Create an engine and connect to the database
engine = create_engine('mssql+pyodbc://DESKTOP-GSR7AST\\SQLEXPRESS/master?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')

# Establish a connection
with engine.begin() as conn:
    # Write your dataframe to SQL and commit the transaction
    df.to_sql('df_orders', con=conn, index=False, if_exists='append')

In [15]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')